# Working with DateTime based dataframes

Pandas is particularly suited for time series data. The author of Pandas, Wes Mckinney, worked at a hedge fund when he brought the functionality of R's dataframes to Python.

In [ ]:
#!pip install hvplot

In [ ]:
import datetime

import pandas as pd
import matplotlib.pyplot as plt
import hvplot.pandas

In [ ]:
%reload_ext postcell
%postcell register

### Load trades data

In [ ]:
%%time
trades_df = pd.read_csv('../../datasets/market_data/trades_2025-09-10_AAPLMSFT_sorted.csv.gz')
trades_df

### Conver the timestamp column to datetime format.

This is necessary to take full advantage time series features.  
Note that Pandas correctly inferred that the timestamps correspond to Sept 10, 2025

In [ ]:
trades_df.sip_timestamp = pd.to_datetime(trades_df.sip_timestamp)
trades_df.head()

### Set the timestamp column as the index

In [ ]:
trades_df.set_index('sip_timestamp', inplace=True)
trades_df.head()

### Benefits of a proper datetime index

Select a specific instance 

In [ ]:
trades_df.loc['2025-09-10 11']

... or lower granularity period

In [ ]:
trades_df.loc['2025-09-10 11:01']

Select date/time ranges

In [ ]:
trades_df.loc['2025-09-10 11:01' : '2025-09-10 11:02']

**Exercise** Find trades a minute before the market opens and minute after (US financial markets open at 9:30). Note that the data is for 2025-09-10

In [ ]:
%%postcell exercise_030_190_a

#type your answer here


`truncate` is a nice convenience function

In [ ]:
trades_df.truncate(before='2025-09-10 9:30', after='2025-09-10 16:00')

### The `resample` method allows analysis at different levels of granularity

Hourly

In [ ]:
trades_df.resample('h').max()

Every 3 minutes

In [ ]:
trades_df.resample('3min').max()

Similar to `groupby`, you can assign a different aggregator to different columns

In [ ]:
trades_df.resample('3min').aggregate({'price':'max', 'size':'mean', 'ticker':'count'})

**Exercise** Find average prices every 30 seconds. See the _offset-alliases_ URL below for a complete list of offset aliases. 

In [ ]:
%%postcell exercise_030_190_b

#type your answer here


Other resampling period identifiers: https://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html#offset-aliases    
Note that the `DateOffset`s defined in the link above are _very_ rich in diversity. 

### Generate data using timestamps

Days between two dates

In [ ]:
pd.date_range(start='2025-09-10', end='2025-10-10')

Weeks between two dates

In [ ]:
pd.date_range(start='2025-09-10', end='2025-10-10', freq='W')

Full list of `freq` aliases available from https://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html#timeseries-offset-aliases

Generate all **business** days between to dates

In [ ]:
pd.bdate_range(start='2025-09-10', end='2025-10-10')

Generate 14 hours from a start date

In [ ]:
pd.date_range(start='2025-01-01', periods=14, freq='h')

**Exercise** Generate a time series containing 30 elements, every 3 hours, starting today at midnight

In [ ]:
%%postcell exercise_030_190_c

#type your answer here


### Window functions

It is common to apply aggregate functions to a time series dataset, not to summarize the data but to smooth it.

In [ ]:
trades_df.loc[trades_df.ticker == 'AAPL'].price.plot.line()
trades_df.loc[trades_df.ticker == 'AAPL'].price.rolling(window=10).mean().plot.line()

In the plot above, we applied a _rolling_ mean to every 10 rows. In trading terms, we created a "moving average."

Apply `max` to 2 rows at a time

In [ ]:
pd.Series([1,2,3,4,5]).rolling(window=2).max()

Same as above, but for 3 rows

In [ ]:
pd.Series([1,2,3,4,5]).rolling(window=3).min()

Let's break down what is in each "window"

In [ ]:
for window in pd.Series([1,2,3,4,5]).rolling(window=3):
    print("="*5)
    print(window)
    print(f'Value of min inside the window: {window.min()}')

When the expression is taken apart, why aren't we getting NaN values?  
For integer windows (like the ones show here), window size corresponds to `min_periods` size. We can over-ride this:

In [ ]:
pd.Series([1,2,3,4,5]).rolling(window=3, min_periods=1).min()

From the docs:
> If an integer, the fixed number of observations used for
    each window.  
>   If a timedelta, str, or offset, the time period of each window. Each
    window will be a variable sized based on the observations included in
    the time-period.

Moving average every 5 seconds (notice the value in `window=`)

In [ ]:
trades_df.loc[trades_df.ticker == 'AAPL'].price.rolling(window='1s').aggregate({'price':'min', 'ticker':'count'})

As we peek into individual windows, we see that when we use time base windows, we no longer get NaN values. Even if there is only a single element in a window, the aggregate is applied to it and there is no min_periods requirement

In [ ]:
COUNTER = 0
for window in trades_df.loc[trades_df.ticker == 'AAPL'].price.rolling(window='1s'):
    print('='*5)
    print(window)

    COUNTER += 1
    if COUNTER > 10: break

More offset aliases found here: https://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html#offset-aliases

**Example: Let's look moving averages crossover**

In financial "Technical analysis," traders often draw two moving averages of different windows in the same chart. When one moving average crosses over another, it indicates a buy or sell opportunity

In [ ]:
# base_trades_df.loc[trades_df.ticker == 'AAPL'].price.plot.line() 
# trades_df.loc[trades_df.ticker == 'AAPL'].price.rolling(window='1min').mean().plot.line()
# trades_df.loc[trades_df.ticker == 'AAPL'].price.rolling(window='10min').mean().plot.line()

Switching to `hvplot` for interactive zoom capability

In [ ]:
#base_plot = trades_df.loc[trades_df.ticker == 'AAPL'].price.hvplot.line() 
min1_plot = trades_df.loc[trades_df.ticker == 'AAPL'].price.rolling(window='1min').mean().hvplot.line()
min10_plot = trades_df.loc[trades_df.ticker == 'AAPL'].price.rolling(window='10min').mean().hvplot.line()

#base_plot * 
(min1_plot * min10_plot)

**exercise** Give `trades_df`, find the moving sum of the `size` column every 30 minutes

In [ ]:
%%postcell exercise_030_190_d

#type your answer here


### Exponentially weighted mean (EWM)
Along with `rolling`, Pandas provides `ewm` which allows more recent observations to be weighted higher than older observations

If `adjusted=False`, then the following formulas apply

`alpha`: Alpha specifies the smoothing factor directly:  $\begin{split}
                y_0 &= x_0\\
                y_t &= (1 - \alpha) y_{t-1} + \alpha x_t,
            \end{split}$  

`span`: Span corresponds to what is commonly called an “N-day EW moving average”: $\alpha = 2 / (span + 1)$, for $span \geq 1$  

`com`: Center of mass has a more physical interpretation from engineering: $\alpha = 1 / (1 + com)$, for $com \geq 0$  

`halflife`: Half-life is the period of time for the exponential weight to reduce to one half: $\alpha = 1 - \exp\left(-\ln(2) / halflife\right)$, for $halflife > 0$

The default for `adjusted` is `True`; however, the calculations are more intuitive when it is `False`

See official docs for more details: https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.ewm.html#pandas.DataFrame.ewm

In [ ]:
pd.Series([1, 2, 3, 4, 5]).ewm(span=2, adjust=False).mean()

In [ ]:
pd.Series([1, 2, 3, 4, 5]).ewm(halflife=2).aggregate(['sum', 'mean'])

In [ ]:
pd.Series([1, 2, 3, 4, 5]).ewm(alpha=.9).aggregate(['sum', 'mean'])